# ECSE 415 Final Project: Classification (SIFT) (35 Points)

Group 10 

April 2022

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

'''
Common google drive for project files and dataset. Mount with this drive.
email: ecse415project2022@gmail.com
password: mcgillecse415
'''

path = "/content/drive/MyDrive/ecse415-project/dataset"
# path = "/content/drive/MyDrive/ecse415-project/dataset_subset"  # SUBSET PATH TO CHANGE FOR ACTUAL TRAINING
# test_path = "/content/drive/MyDrive/ecse415-project/dataset_tyler_test"

Mounted at /content/drive


In [ ]:
#The version of cv2 needed is 4.4. As mentioned in Tutorial 4, the following command must be run to install the appropriate cv2 library.
!pip install opencv-python==4.4.0.44
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2
import os
import pickle

from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

## Extract Images

In [ ]:
def images_to_arr(dataset_path, resize_shape=(100, 100)):


  veh_path = os.path.join(dataset_path, "vehicle")
  non_veh_path = os.path.join(dataset_path, "non_vehicle")

  veh_listdir = os.listdir(veh_path)
  non_veh_listdir = os.listdir(non_veh_path)
  
  veh_images = []
  non_veh_images = []

  for f in veh_listdir:
    img = cv2.imread(os.path.join(veh_path, f))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = cv2.resize(img, resize_shape)
    veh_images.append(img)
  
  for f in non_veh_listdir:
    img = cv2.imread(os.path.join(non_veh_path, f))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = cv2.resize(img, resize_shape)
    non_veh_images.append(img)

  # concat vehicles and non-vehicles
  images = np.concatenate([np.array(veh_images), np.array(non_veh_images)])
  labels = np.concatenate([np.full(len(veh_listdir), 1.), np.full(len(non_veh_listdir), 0.)])

  return images, labels

In [ ]:
# DANGER: only run if the files don't already exist
images_0000, labels_0000 = images_to_arr(path + "/0000")
np.save(os.path.join(path, 'images_0000'), images_0000)
np.save(os.path.join(path, 'labels_0000'), labels_0000)

images_0001, labels_0001 = images_to_arr(path + "/0001")
np.save(os.path.join(path, 'images_0001'), images_0001)
np.save(os.path.join(path, 'labels_0001'), labels_0001)

images_0002, labels_0002 = images_to_arr(path + "/0002")
np.save(os.path.join(path, 'images_0002'), images_0002)
np.save(os.path.join(path, 'labels_0002'), labels_0002)

images_0003, labels_0003 = images_to_arr(path + "/0003")
np.save(os.path.join(path, 'images_0003'), images_0003)
np.save(os.path.join(path, 'labels_0003'), labels_0003)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
images_0000 = np.load(os.path.join(path, 'images_0000.npy'), allow_pickle=True)
images_0001 = np.load(os.path.join(path, 'images_0001.npy'), allow_pickle=True)
images_0002 = np.load(os.path.join(path, 'images_0002.npy'), allow_pickle=True)
images_0003 = np.load(os.path.join(path, 'images_0003.npy'), allow_pickle=True)
labels_0000 = np.load(os.path.join(path, 'labels_0000.npy'), allow_pickle=True)
labels_0001 = np.load(os.path.join(path, 'labels_0001.npy'), allow_pickle=True)
labels_0002 = np.load(os.path.join(path, 'labels_0002.npy'), allow_pickle=True)
labels_0003 = np.load(os.path.join(path, 'labels_0003.npy'), allow_pickle=True)
extra_images_0000 = np.load(os.path.join(path, 'extra_images_0000.npy'), allow_pickle=True)
extra_images_0001 = np.load(os.path.join(path, 'extra_images_0001.npy'), allow_pickle=True)
extra_images_0002 = np.load(os.path.join(path, 'extra_images_0002.npy'), allow_pickle=True)
extra_images_0003 = np.load(os.path.join(path, 'extra_images_0003.npy'), allow_pickle=True)
extra_labels_0000 = np.load(os.path.join(path, 'extra_labels_0000.npy'), allow_pickle=True)
extra_labels_0001 = np.load(os.path.join(path, 'extra_labels_0001.npy'), allow_pickle=True)
extra_labels_0002 = np.load(os.path.join(path, 'extra_labels_0002.npy'), allow_pickle=True)
extra_labels_0003 = np.load(os.path.join(path, 'extra_labels_0003.npy'), allow_pickle=True)

## Classifier


#### Description of Idea

SVM Classifier Idea using SIFT:

1. extract SIFT features for each training sample

      -- optionally select a random number N of keypoints for each image

2. compute K-means over all the N keypoints from each image (total of N*#images)
  
      -- optionally use mean-shift to do this to dynamically determine clusters
  
3. re-extract SIFT features for each training sample and create a histogram based on which clusters the features fall into. This will give us a vector of leangth K for each image with each value in the vector indicating the weight of that cluster in that image 

      -- this vector should be L1 normalized

4. build an SVM classifier that takes in that K-vector for each image

### 1 - SIFT Features

In [ ]:
NUM_DESC = 50

def extract_all_descriptors(images, labels, num_desc):
  """
  Given a list of images with their labels, this function returns 2 numpy arrays.
  The first array will be a list of all descriptors from each image. We do however
  limit each image to only provide ${num_desc} number of descriptors.
  """

  all_desc = []
  descriptors = np.empty((0, 128))
  for i, img in enumerate(images):
    # use gray image to get keypoints?
    # gray= cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    feature_extractor = cv2.SIFT_create()
    kp, desc = feature_extractor.detectAndCompute(img, None)
    if desc is not None:
      all_desc.append((desc, labels[i]))
      desc = desc[:num_desc]
      descriptors = np.concatenate((descriptors, desc))
  return descriptors, np.array(all_desc)

In [ ]:
def get_descriptors(all_descriptors, max_desc_num):
  """
  This function will use the exported descriptors to extract a list of all
  desciptors from a dataset such that each image only provides a max number of
  descriptors (max_desc_num)
  """
  descriptors = list(all_descriptors[0][0][:max_desc_num])
  for i, d in enumerate(all_descriptors[1:]):
    if i % 300 == 0:
      print(i)
    descriptors = descriptors + list(d[0][:max_desc_num])
  return np.array(descriptors)


In [ ]:
# # DANGER: only run if the files don't already exist - takes a long time to run (30 min)

# descriptors_0000, all_descriptors_0000 = extract_all_descriptors(images_0000, labels_0000, NUM_DESC)
# np.save(os.path.join(path, 'descriptors_0000'), descriptors_0000)
# np.save(os.path.join(path, 'all_descriptors_0000'), all_descriptors_0000)
# descriptors_0001, all_descriptors_0001 = extract_all_descriptors(images_0001, labels_0001, NUM_DESC)
# np.save(os.path.join(path, 'descriptors_0001'), descriptors_0001)
# np.save(os.path.join(path, 'all_descriptors_0001'), all_descriptors_0001)
# descriptors_0002, all_descriptors_0002 = extract_all_descriptors(images_0002, labels_0002, NUM_DESC)
# np.save(os.path.join(path, 'descriptors_0002'), descriptors_0002)
# np.save(os.path.join(path, 'all_descriptors_0002'), all_descriptors_0002)

In [ ]:
# DANGER: only run if the files don't already exist - takes a long time to run (30 min)

extra_descriptors_0000, extra_all_descriptors_0000 = extract_all_descriptors(extra_images_0000, extra_labels_0000, NUM_DESC)
np.save(os.path.join(path, 'extra_descriptors_0000'), extra_descriptors_0000)
np.save(os.path.join(path, 'extra_all_descriptors_0000'), extra_all_descriptors_0000)
extra_descriptors_0001, extra_all_descriptors_0001 = extract_all_descriptors(extra_images_0001, extra_labels_0001, NUM_DESC)
np.save(os.path.join(path, 'extra_descriptors_0001'), extra_descriptors_0001)
np.save(os.path.join(path, 'extra_all_descriptors_0001'), extra_all_descriptors_0001)
extra_descriptors_0002, extra_all_descriptors_0002 = extract_all_descriptors(extra_images_0002, extra_labels_0002, NUM_DESC)
np.save(os.path.join(path, 'extra_descriptors_0002'), extra_descriptors_0002)
np.save(os.path.join(path, 'extra_all_descriptors_0002'), extra_all_descriptors_0002)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
descriptors_0000 = np.load(os.path.join(path, 'descriptors_0000.npy'), allow_pickle=True)
descriptors_0001 = np.load(os.path.join(path, 'descriptors_0001.npy'), allow_pickle=True)
descriptors_0002 = np.load(os.path.join(path, 'descriptors_0002.npy'), allow_pickle=True)

all_descriptors_0000 = np.load(os.path.join(path, 'all_descriptors_0000.npy'), allow_pickle=True)
all_descriptors_0001 = np.load(os.path.join(path, 'all_descriptors_0001.npy'), allow_pickle=True)
all_descriptors_0002 = np.load(os.path.join(path, 'all_descriptors_0002.npy'), allow_pickle=True)

### 2 - KMeans

In [ ]:
NUM_CLUSTERS = 250

In [ ]:
# descriptors_0000 = get_descriptors(all_descriptors_0000, 100)
# mask = all_descriptors_0001[:, 1] == 0.
# pos_desc = all_descriptors_0001[~mask]
# neg_desc = all_descriptors_0001[mask]
# idx = np.random.randint(neg_desc.shape[0], size=2888)
# neg_desc = neg_desc[idx, :]
# print(pos_desc.shape, neg_desc.shape)
# all_desc = np.concatenate((pos_desc, neg_desc))
# print(all_desc.shape)
# descriptors_0001 = get_descriptors(all_desc, 100)

In [ ]:
train_descriptors_0_1 = np.concatenate((descriptors_0000, descriptors_0001, extra_descriptors_0000, extra_descriptors_0001))

In [ ]:
# Training KMeans takes about 10 min fro 10 clusters


# Validation kmeans
kmeans_00_01 = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(train_descriptors_0_1)
# kmeans_00_02 = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(np.concatenate((descriptors_0000, descriptors_0002)))
# kmeans_01_02 = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(np.concatenate((descriptors_0001, descriptors_0002)))

# # Will be used as final model - test on 03
# kmeans_00_01_02 = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(np.concatenate((descriptors_0000, descriptors_0001, descriptors_0002)))

In [ ]:
with open(os.path.join(path, f"kmeans_00_01_75_{NUM_CLUSTERS}.pkl"), "wb") as f:
    pickle.dump(kmeans_00_01, f)

In [ ]:
with open(os.path.join(path, f"kmeans_00_01_{NUM_CLUSTERS}.pkl"), "rb") as f:
    kmeans_00_01 = pickle.load(f)

### 3 - Validation Tests

#### Helper Methods

In [ ]:
'''
Create bin for each sample based on what clusters its sift descriptors belong to.
Normalize all bins. These will be used as training samples.
'''

def desc_to_norm_cluster_bin(kmeans, desc, num_clusters):
  """
  This function will take all the descriptors in desc and find the 
  closest clusters for each one using the trained kmeans model passed.
  It will produce a bincount of how many times each cluster is seen and normalize
  this.
  So no matter how many descriptors are passed (we should keep this constant),
  we will output a normalized vector of length NUM_CLUSTERS.
  """
  clusters = kmeans.predict(list(desc))
  bin = np.bincount(clusters)
  if len(bin) < num_clusters:
    bin = np.pad(bin, (0, num_clusters - len(bin)), 'constant')
  norm_bin = bin / np.sum(bin)
  return norm_bin


def descriptors_to_bins(kmeans, descriptors, num_clusters):
  """
  This function takes in a kmeans model as well as a list containing tuples (descriptors).
  The tuple has 2 elements, the first is the descriptors for an image, and the second
  is a label.

  It will produce a dataset where the descriptors of each image are transformed
  into normalized bin vector of size NUM_CLUSTERS
  """

  X = np.empty((len(descriptors), num_clusters))
  y = np.empty(len(descriptors), dtype=float)
  for i, (desc, label) in enumerate(descriptors):
    desc = desc.astype(float)
    norm_bin = desc_to_norm_cluster_bin(kmeans, desc, num_clusters)

    X[i] = norm_bin
    y[i] = label
  return X, y

In [ ]:
def validation(images, labels, kmeans, model, num_clusters):
  """
  Give a set of images, labels a kmeans model as well as a general model
  that accept vectors of dimension ${num_clusters}, this method will use the 
  model to make predictions on all the images and return statistics on its
  performance.
  """

  predictions = np.empty(len(images), dtype=float)
  count = 0
  for i, img in enumerate(images):
    feature_extractor = cv2.SIFT_create()
    kp, desc = feature_extractor.detectAndCompute(img, None)
    if desc is None:
      predictions[i] = 0.
      count+=1
      continue
    desc = desc.astype(float)
    norm_bin = desc_to_norm_cluster_bin(kmeans, desc, num_clusters)
    predictions[i] = model.predict([norm_bin])

  acc = np.sum(predictions == labels) / len(labels)
  precision, recall, _, _ = precision_recall_fscore_support(labels, predictions, average='binary')
  print(count)
  return acc, precision, recall, predictions

In [ ]:
def train_and_predict(test_images, labels, train_descriptors, num_clusters, model, kmeans):
  X,y = descriptors_to_bins(kmeans, train_descriptors, NUM_CLUSTERS)
  model.fit(X,y)
  acc, precision, recall, predictions = validation(test_images, labels, kmeans, model, num_clusters)
  return model, acc, precision, recall, predictions

#### Validation Tests

##### SVM

In [ ]:
# Validation Test 1: Trained on 0000 and 0001 - Test on 0002

clf_01 = make_pipeline(MinMaxScaler(), SVC(kernel='rbf', degree=6, random_state=0, tol=1e-5, max_iter=30000, C=1))
model_1, acc1, p1, r1, pred1 = train_and_predict(
                  images_0002, 
                  labels_0002, 
                  np.concatenate((all_descriptors_0000, all_descriptors_0001)), 
                  NUM_CLUSTERS, 
                  clf_01, 
                  kmeans_00_01
                  )

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


104


In [ ]:
 # C = 100000, kmeans = 200 clusters
print(acc1, p1, r1)
tn, fp, fn, tp = confusion_matrix(labels_0002, pred1).ravel()
print(f"tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}")

0.3489749430523918 0.20500595947556616 0.7835990888382688
tn: 844, fp: 2668, fn: 190, tp: 688


In [ ]:
# C = 10000, kmeans = 500 clusters
print(acc1, p1, r1)
tn, fp, fn, tp = confusion_matrix(labels_0002, pred1).ravel()
print(f"tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}")

0.6266514806378132 0.24445936870382806 0.4145785876993166
tn: 2387, fp: 1125, fn: 514, tp: 364


In [ ]:
# Validation Test 1: Trained on 0000 and 0001 - Test on 0002

clf_01 = make_pipeline(StandardScaler(), SVC(kernel='linear', degree=6, random_state=0, tol=1e-5, max_iter=30000, C=10000))
model_1, acc1, p1, r1, pred1 = train_and_predict(
                  images_0002, 
                  labels_0002, 
                  np.concatenate((all_descriptors_0000, all_descriptors_0001)), 
                  NUM_CLUSTERS, 
                  clf_01,  
                  kmeans_00_01
                  )

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=30000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


104


In [ ]:
print(acc1, p1, r1)
tn, fp, fn, tp = confusion_matrix(labels_0002, pred1).ravel()
print(f"tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}")

0.3548974943052392 0.20730976632714201 0.7881548974943052
tn: 866, fp: 2646, fn: 186, tp: 692


In [ ]:
# Validation Test 2: Trained on 0000 and 0002 - Test on 0001

clf_02 = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=0, tol=1e-5, max_iter=30000))
model_2, acc2, p2, r2, pred2 = train_and_predict(
                  images_0001, 
                  labels_0001, 
                  np.concatenate((all_descriptors_0000, all_descriptors_0002)), 
                  NUM_CLUSTERS, 
                  clf_02, 
                  kmeans_00_02
                  )

In [ ]:
# Validation Test 3: Trained on 0001 and 0002 - Test on 0000

clf_12 = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=0, tol=1e-5, max_iter=30000))
model_3, acc3, p3, r3, pred3 = train_and_predict(
                  images_0000, 
                  labels_0000, 
                  np.concatenate((all_descriptors_0001, all_descriptors_0002)), 
                  NUM_CLUSTERS, 
                  clf_12, 
                  kmeans_01_02
                  )

##### Logistic Regression

In [ ]:
# Validation Test 1: Trained on 0000 and 0001 - Test on 0002

clf_lr_01 = LogisticRegression(random_state=0)
model_1, acc1, p1, r1, pred1 = train_and_predict(
                  images_0002, 
                  labels_0002, 
                  np.concatenate((all_descriptors_0000, all_descriptors_0001)), 
                  NUM_CLUSTERS, 
                  clf_01, 
                  kmeans_00_01
                  )

In [ ]:
# Validation Test 2: Trained on 0000 and 0002 - Test on 0001

clf_02 = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5, max_iter=30000))
model_2, acc2, p2, r2, pred2 = train_and_predict(
                  images_0001, 
                  labels_0001, 
                  np.concatenate((all_descriptors_0000, all_descriptors_0002)), 
                  NUM_CLUSTERS, 
                  clf_02, 
                  kmeans_00_02
                  )

In [ ]:
# Validation Test 3: Trained on 0001 and 0002 - Test on 0000

clf_12 = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5, max_iter=30000))
model_3, acc3, p3, r3, pred3 = train_and_predict(
                  images_0000, 
                  labels_0000, 
                  np.concatenate((all_descriptors_0001, all_descriptors_0002)), 
                  NUM_CLUSTERS, 
                  clf_12, 
                  kmeans_01_02
                  )